In [1]:
import pandas as pd
import random as rnd 
import mysql.connector
from mysql.connector import Error
import plotly
import json
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pymysql
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
import networkx as nx
from networkx_viewer import Viewer
from functools import reduce
from scipy.spatial.distance import pdist, squareform, cdist
import jgraph
from cyjupyter import Cytoscape
from scipy.stats import kendalltau
from scipy.stats import weightedtau
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import seaborn as sn
from scipy import stats

In [2]:
info_dict = {'descriptor_domain':['abilites','work_context','interests',
                               'knowledge','skills','work_styles','work_values','gwas','job_zones'],
            'starter':['IM,LV',None,'OI',None,None,'IM','EX',None,None],
            'change':[None,'CX',None,'IM, LV','IM, LV',None,None,'IM, LV','JZ']}
df_info = pd.DataFrame(data = info_dict)
df_info

,descriptor_domain,starter,change
0,abilites,"IM,LV",None
1,work_context,None,CX
2,interests,OI,None
3,knowledge,None,"IM, LV"
4,skills,None,"IM, LV"
5,work_styles,IM,None
6,work_values,EX,None
7,gwas,None,"IM, LV"
8,job_zones,None,JZ


In [3]:
connection = mysql.connector.connect(host='127.0.0.1',
                                         database='employment',
                                         user='root',
                                         password='thanhn123')

In [4]:
def get_data(connection, descriptor_domain,LV_only = False, filt = False):
    '''
    Return df in the SOCs-decriptor format of a specific domain 
    descriptor_domeain:
        work context
        knowledge
        skills
        gwas
        job_zones, filter
        Abilities, filter
    
    '''
    df= pd.DataFrame(data=None)
    if descriptor_domain == 'work_context':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_context
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'CX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'job_zones':
        if filt:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
            df = df[df['job_zone'].isin(val)]
        else:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
    elif descriptor_domain == 'abilities':
        if filt:
#             df= pd.read_sql('''
#             SELECT title, element_name, element_id,data_value FROM employment.''' + descriptor_domain + '''
#             LEFT JOIN employment.content_model_reference USING (element_id) 
#             LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
#             df = df[df['element_id'].contains(val)]
            df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE element_id LIKE\''''+ val +'''%\'; ''', con = connection)
            
        else:
            df= pd.read_sql('''
            SELECT title, element_name,element_id, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_values':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_values
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'EX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'interests':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.interests
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'OI';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_styles':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_styles
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'IM';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'name':
        df= pd.read_sql('''
            SELECT * FROM employment.occupation_data;''', con = connection)
    else:
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        if LV_only:
            df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE \"LV\";''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    return df

In [5]:
def distance_table(df, jz=False):
    
#     squareform(pdist(df.iloc[:, 1:])), columns=df.title.unique(), index=df.title.unique()
    dist = []
    name_dict = {}
    if jz:
        name_dict = df['title'].sort_values().reset_index()['title'].to_dict()
        dist = pdist(df[df.columns[1:]],'euclidean')
    else:
        for i in range(len(df.index)):
            name_dict[i] = df.index[i]
        dist = pdist(df,'euclidean') #Euclidean distance for 1XN
    df_dist = pd.DataFrame(squareform(dist)) #Euclidean distance for NxN
#     return df_dist
    columns = list(df_dist.index) 
    for i in range(len(df.index)): #convert 0 values to NaN
        df_dist.loc[i,i] = np.NaN
    mean = df_dist.mean().mean()
    std = df_dist.stack().std() 

    for col in columns: #Z-score calculated 
        df_dist[col] = (df_dist[col] - mean)/std
#     return df_dist
    return df_dist.rename(columns = name_dict, index = name_dict)

In [6]:
df_ab = get_data(connection,'abilities')
df_in = get_data(connection,'interests')
df_ws = get_data(connection,'work_styles')
df_wv = get_data(connection, 'work_values')
df_wc = get_data(connection,'work_context')
df_knowledge = get_data(connection,'knowledge')
df_gwas = get_data(connection,'work_activities')
df_jz = get_data(connection,'job_zones')
df_jz = df_jz[df_jz['title'] != 'Legislators']
df_skills = get_data(connection,'skills')

In [7]:
wc_dist = distance_table(df_wc)
knowledge_dist = distance_table(df_knowledge)
skills_dist = distance_table(df_skills)
gwas_dist = distance_table(df_gwas)
jz_dist = distance_table(df_jz,True)
wv_dist = distance_table(df_wv)
in_dist = distance_table(df_in)
ab_dist = distance_table(df_ab)
ws_dist = distance_table(df_ws)

In [8]:
# df_name = pd.read_sql('''
#             SELECT onetsoc_code,title FROM employment.occupation_data;''', con = connection)
# name_dict = df_name.set_index('onetsoc_code').to_dict()
# name_dict_cps = {}
# for val in name_dict['title']:
# #     print(val[:-3])
# #     break
#     name_dict_cps[val[:-3]] = name_dict['title'][val] 

In [9]:
df_name = pd.read_csv('../CPS Job Changes/2010_to_2018_SOC_Crosswalk.csv')
df_name = df_name[['2018 SOC Code','2018 SOC Title']]
df_name = df_name.set_index('2018 SOC Code').to_dict()
df_name = df_name['2018 SOC Title']

In [10]:
df_cps = pd.read_csv('../CPS Job Changes/JobChanges_2011to19.csv')

In [11]:
df_cps.drop(columns =['in_grp','in_minor_grp','in_broad_occ'])

,ONET18_SOC_LY,ONET18_SOC,ONET18_Title,ONET18_Title_LY,count,pct_tot,count_socly,pct_socly,employment_projection_NewJob,wage_change_BetweenJobs
0,11-1011,11-1021,General and Operations Managers,Chief Executives,327891.330000,0.196284,644769.200000,50.854062,5.8,-45.365359
1,11-1011,11-1031,Legislators,Chief Executives,4069.996250,0.002436,644769.200000,0.631233,6.2,-84.131579
2,11-1011,11-2021,Marketing Managers,Chief Executives,5621.145000,0.003365,644769.200000,0.871807,6.7,-25.811908
3,11-1011,11-2022,Sales Managers,Chief Executives,5621.145000,0.003365,644769.200000,0.871807,3.5,-31.337393
4,11-1011,11-3012,Administrative Services Managers,Chief Executives,1832.805000,0.001097,644769.200000,0.284258,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
115312,53-7199,53-7065,Stockers and Order Fillers,"Material Moving Workers, All Other",4878.226667,0.002920,43481.721667,11.219028,0.7,-13.874346
115313,53-7199,53-7071,Gas Compressor and Gas Pumping Station Operators,"Material Moving Workers, All Other",35.795833,0.000021,43481.721667,0.082324,-2.1,108.180628
115314,53-7199,53-7072,"Pump Operators, Except Wellhead Pumpers","Material Moving Workers, All Other",35.795833,0.000021,43481.721667,0.082324,9.7,41.688482
115315,53-7199,53-7073,Wellhead Pumpers,"Material Moving Workers, All Other",35.795833,0.000021,43481.721667,0.082324,4.2,73.167539


In [12]:
df_all = pd.read_csv('../CPS Job Changes/GoodnessIndicators_allCombinations.csv')
df_all = df_all.replace({'SOC2018_origin':df_name, 'SOC2018_destination':df_name})

In [14]:
df_all

,SOC2018_destination,SOC2018_origin,median_h_wage_o,median_h_wage_d,median_h_wage_d.1,employment_projection_NewJob,wage_change_BetweenJobs
0,Chief Executives,Correctional Officers and Jailers,21.720000,-10.0,88.68,-10.0,308.287293
1,Chief Executives,"Environmental Science Teachers, Postsecondary",39.629808,-10.0,88.68,-10.0,123.770957
2,Chief Executives,School Psychologists,NaN,-10.0,88.68,-10.0,NaN
3,Chief Executives,"Human Resources Assistants, Except Payroll and...",19.920000,-10.0,88.68,-10.0,345.180723
4,Chief Executives,Human Resources Specialists,29.770000,-10.0,88.68,-10.0,197.883776
...,...,...,...,...,...,...,...
752635,NaN,39-7010,13.270000,NaN,NaN,NaN,NaN
752636,NaN,53-6098,17.790000,NaN,NaN,NaN,NaN
752637,NaN,29-1228,99.280000,NaN,NaN,NaN,NaN
752638,NaN,15-1256,51.690000,NaN,NaN,NaN,NaN


In [34]:
df_all[(df_all['SOC2018_origin'] == 'Chief Executives') & (df_all['SOC2018_destination'] == 'Correctional Officers and Jailers')]

,SOC2018_destination,SOC2018_origin,median_h_wage_o,median_h_wage_d,median_h_wage_d.1,employment_projection_NewJob,wage_change_BetweenJobs
357489,Correctional Officers and Jailers,Chief Executives,88.68,-7.5,21.72,-7.5,-75.507442


In [35]:
df_cps[(df_cps['ONET18_Title_LY'] == 'Chief Executives') & (df_cps['ONET18_Title'] == 'Correctional Officers and Jailers')]

,ONET18_SOC_LY,ONET18_SOC,ONET18_Title,ONET18_Title_LY,count,pct_tot,count_socly,pct_socly,in_grp,in_minor_grp,in_broad_occ,employment_projection_NewJob,wage_change_BetweenJobs


## drop values based on <0

In [19]:
def exact_difference(df):
    '''
    return adjacency matrix difference. Difference is based on column - row
    zeroed out 
    '''
    ini_array1 = df.values - df.values[:, None]
    
    arr = ini_array1[ini_array1 <0] = 0
    arr = np.sqrt(np.mean(np.power(ini_array1,2),axis =2))
#     return arr
    new_df = pd.concat((df.reset_index()['title'], pd.DataFrame(arr, columns=df.reset_index()['title'])), axis=1)
    return new_df

In [20]:
def get_df_comparison(df_onet,df_cps,good_dict,occ,top):
    '''
    return df of comparisons of different occupations
    
    '''
    comparison = {'ONET_occ_before':[],'ONET_occ_after':[],'ONET_employment_proj' : [],
                       'ONET_wage_change':[],'CPS_occ_before': [],'CPS_occ_after': [],'CPS_employment_proj' : [],'CPS_wage_change':[]}
    
    df_cps_iter = df_cps[df_cps['ONET18_Title_LY'] == occ].sort_values(by = 'pct_tot', ascending = False)[0:top]
    df_onet_iter = df_onet.loc[occ].sort_values()[1:top+1].to_frame()
    
    for row in df_cps_iter.iterrows():
        if (occ,row[1][2]) not in good_dict:
            comparison['CPS_occ_before'].append(occ)
            comparison['CPS_occ_after'].append(row[1][2])
            comparison['CPS_employment_proj'].append(None)
            comparison['CPS_wage_change'].append(None)
        else:
            comparison['CPS_occ_before'].append(occ)
            comparison['CPS_occ_after'].append(row[1][2])
            comparison['CPS_employment_proj'].append(good_dict[(occ,row[1][2])][0])
            comparison['CPS_wage_change'].append(good_dict[(occ,row[1][2])][1])
    for row in df_onet_iter.iterrows():
        if (occ,row[0]) not in good_dict:
            comparison['ONET_occ_before'].append(occ)
            comparison['ONET_occ_after'].append(row[0])
            comparison['ONET_employment_proj'].append(None)
            comparison['ONET_wage_change'].append(None)
        else:
            comparison['ONET_occ_before'].append(occ)
            comparison['ONET_occ_after'].append(row[0])
            comparison['ONET_employment_proj'].append(good_dict[(occ,row[0])][0])
            comparison['ONET_wage_change'].append(good_dict[(occ,row[0])][1])
    return pd.DataFrame(comparison)


In [21]:
difference_abilites = exact_difference(df_ab)
# difference_in = exact_difference(df_in)
# difference_ws = exact_difference(df_ws)
# difference_wv =exact_difference(df_wv)
# difference_wc =exact_difference(df_wc)
# difference_knowledge =exact_difference(df_knowledge)
difference_gwas =exact_difference(df_gwas)
# difference_jz =exact_difference(df_jz)
difference_skills =exact_difference(df_skills)

In [22]:
difference_skills.index = difference_skills.title 
difference_abilites.index = difference_abilites.title 
difference_gwas.index = difference_gwas.title 

difference_skills = difference_skills.drop(['title'],axis =1)
difference_abilites = difference_abilites.drop(['title'],axis =1)
difference_gwas = difference_gwas.drop(['title'],axis =1)

## Not dropping values

In [23]:
df_concat = pd.concat([ab_dist,skills_dist,gwas_dist]).groupby(level =0).mean()

## dropping values

In [24]:
df_concat_exact_difference = pd.concat([difference_gwas,difference_skills,difference_abilites]).groupby(level =0).mean()

## CPS data

In [25]:
# df_cps[:10]

In [26]:
# df_all

In [27]:
# for row in df_cps.iterrows():
#     print(row)
#     print(row[1][0],row[1][1])
#     print(row[1][-2],row[1][-1])
#     break

In [28]:
# for row in df_all.iterrows():
#     print(row)
#     print(row[1][0],row[1][1])
#     print(row[1][-2],row[1][-1])
#     break

In [29]:
good_dict = {}
for row in df_cps.iterrows():

    #(ONET18_Title_LY,ONET18_Title) = [employment_projection_NewJob,wage_change_BetweenJobs]
    good_dict[(row[1][3],row[1][2])] = [row[1][-2],row[1][-1]]

In [30]:
good_dict[('Chief Executives', 'General and Operations Managers')]

[5.8, -45.36535859269279]

## all data

In [42]:
# df_all = pd.read_csv('../CPS Job Changes/GoodnessIndicators_allCombinations.csv')
# df_all = df_all.replace({'SOC2018_origin':df_name, 'SOC2018_destination':df_name})
pd.read_csv('../CPS Job Changes/GoodnessIndicators_allCombinations.csv')

,SOC2018_destination,SOC2018_origin,median_h_wage_o,median_h_wage_d,median_h_wage_d.1,employment_projection_NewJob,wage_change_BetweenJobs
0,11-1011,33-3012,21.720000,-10.0,88.68,-10.0,308.287293
1,11-1011,25-1053,39.629808,-10.0,88.68,-10.0,123.770957
2,11-1011,19-3034,NaN,-10.0,88.68,-10.0,NaN
3,11-1011,43-4161,19.920000,-10.0,88.68,-10.0,345.180723
4,11-1011,13-1071,29.770000,-10.0,88.68,-10.0,197.883776
...,...,...,...,...,...,...,...
752635,NaN,39-7010,13.270000,NaN,NaN,NaN,NaN
752636,NaN,53-6098,17.790000,NaN,NaN,NaN,NaN
752637,NaN,29-1228,99.280000,NaN,NaN,NaN,NaN
752638,NaN,15-1256,51.690000,NaN,NaN,NaN,NaN


In [45]:
# df_all = pd.read_csv('../CPS Job Changes/GoodnessIndicators_allCombinations.csv')
# df_all = df_all.replace({'SOC2018_origin':df_name, 'SOC2018_destination':df_name})
good_dict_new = {}
i =0
for row in df_all.iterrows():
    good_dict_new[(row[1][1],row[1][0])] = [row[1][-2],row[1][-1]]

In [46]:
good_dict_new[('Chief Executives', 'General and Operations Managers')]

[5.8, -45.36535859269279]

In [47]:
# good_dict_new[('Chief Executives', 'General and Operations Managers')]

## No Drop

In [48]:
get_df_comparison(df_concat,df_cps,good_dict_new,'Registered Nurses',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,Registered Nurses,Licensed Practical and Licensed Vocational Nurses,9.1,-35.215664,Registered Nurses,Physical Therapists,18.2,22.020431
1,Registered Nurses,Physical Therapists,18.2,22.020431,Registered Nurses,Respiratory Therapists,19.4,-16.345062
2,Registered Nurses,Athletic Trainers,16.2,-33.914695,Registered Nurses,"Therapists, All Other",14.6,-28.178207
3,Registered Nurses,Nurse Practitioners,52.4,49.829739,Registered Nurses,Exercise Physiologists,11.3,-32.917140
4,Registered Nurses,Midwives,NaN,NaN,Registered Nurses,Speech-Language Pathologists,24.9,7.945516
5,Registered Nurses,Exercise Physiologists,11.3,-32.917140,Registered Nurses,Occupational Therapists,15.9,15.891033
6,Registered Nurses,Veterinarians,15.9,30.249716,Registered Nurses,Nurse Midwives,11.6,43.303065
7,Registered Nurses,Nurse Midwives,11.6,43.303065,Registered Nurses,Nurse Practitioners,52.4,49.829739
8,Registered Nurses,Physician Assistants,31.3,53.149830,Registered Nurses,Licensed Practical and Licensed Vocational Nurses,9.1,-35.215664
9,Registered Nurses,Urologists,NaN,NaN,Registered Nurses,Nurse Anesthetists,13.7,138.450624


In [49]:
get_df_comparison(df_concat,df_cps,good_dict_new,'Retail Salespersons',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,Retail Salespersons,"Hotel, Motel, and Resort Desk Clerks",-6.9,-3.130148,Retail Salespersons,Cashiers,-7.4,-6.342669
1,Retail Salespersons,Cashiers,-7.4,-6.342669,Retail Salespersons,Gambling Change Persons and Booth Cashiers,2.5,1.729819
2,Retail Salespersons,Counter and Rental Clerks,1.1,14.168040,Retail Salespersons,Insurance Sales Agents,5.5,101.729819
3,Retail Salespersons,Waiters and Waitresses,3.7,-9.390445,Retail Salespersons,"Sales Representatives of Services, Except Adve...",6.0,122.322900
4,Retail Salespersons,Postal Service Clerks,-13.5,91.350906,Retail Salespersons,"Sales and Related Workers, All Other",1.9,26.029654
5,Retail Salespersons,"Stock Clerks, Sales Floor",NaN,NaN,Retail Salespersons,"Sales Representatives, Wholesale and Manufactu...",4.1,220.840198
6,Retail Salespersons,Tellers,-15.3,23.723229,Retail Salespersons,"Sales Representatives, Wholesale and Manufactu...",0.9,137.314662
7,Retail Salespersons,Bartenders,5.9,-6.177924,Retail Salespersons,Real Estate Sales Agents,2.3,93.822076
8,Retail Salespersons,"Sales Representatives, Wholesale and Manufactu...",0.9,137.314662,Retail Salespersons,Real Estate Brokers,2.6,136.490939
9,Retail Salespersons,Reservation and Transportation Ticket Agents a...,-2.9,51.976936,Retail Salespersons,Receptionists and Information Clerks,3.6,19.028007


In [50]:
get_df_comparison(df_concat,df_cps,good_dict_new,'Janitors and Cleaners, Except Maids and Housekeeping Cleaners',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,"Janitors and Cleaners, Except Maids and Housek...",Tapers,-4.0,115.314632,"Janitors and Cleaners, Except Maids and Housek...",Maids and Housekeeping Cleaners,-0.3,-9.401061
1,"Janitors and Cleaners, Except Maids and Housek...",Laundry and Dry-Cleaning Workers,0.2,-11.751327,"Janitors and Cleaners, Except Maids and Housek...",Landscaping and Groundskeeping Workers,10.1,10.917362
2,"Janitors and Cleaners, Except Maids and Housek...",Slaughterers and Meat Packers,3.3,6.520091,"Janitors and Cleaners, Except Maids and Housek...","Pesticide Handlers, Sprayers, and Applicators,...",8.3,30.629265
3,"Janitors and Cleaners, Except Maids and Housek...",Cleaners of Vehicles and Equipment,3.6,-5.989386,"Janitors and Cleaners, Except Maids and Housek...","Grounds Maintenance Workers, All Other",6.5,16.982563
4,"Janitors and Cleaners, Except Maids and Housek...","Helpers--Painters, Paperhangers, Plasterers, a...",1.9,14.253222,"Janitors and Cleaners, Except Maids and Housek...",Tree Trimmers and Pruners,10.7,45.716452
5,"Janitors and Cleaners, Except Maids and Housek...",Shoe and Leather Workers and Repairers,-10.3,7.733131,"Janitors and Cleaners, Except Maids and Housek...",Pest Control Workers,8.5,36.087945
6,"Janitors and Cleaners, Except Maids and Housek...",Solderers and Brazers,NaN,NaN,"Janitors and Cleaners, Except Maids and Housek...",Construction Laborers,5.4,34.344200
7,"Janitors and Cleaners, Except Maids and Housek...",Paperhangers,-0.8,47.687642,"Janitors and Cleaners, Except Maids and Housek...",Retail Salespersons,-0.6,-7.960576
8,"Janitors and Cleaners, Except Maids and Housek...","Pressers, Textile, Garment, and Related Materials",-18.9,-11.827142,"Janitors and Cleaners, Except Maids and Housek...",Childcare Workers,1.7,-11.675512
9,"Janitors and Cleaners, Except Maids and Housek...","Postal Service Mail Sorters, Processors, and P...",-17.8,119.181198,"Janitors and Cleaners, Except Maids and Housek...","Laborers and Freight, Stock, and Material Move...",4.2,7.581501


## Drop <0

In [51]:
get_df_comparison(df_concat_exact_difference,df_cps,good_dict_new,'Registered Nurses',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,Registered Nurses,Teacher Assistants,NaN,NaN,Registered Nurses,Physical Therapists,18.2,22.020431
1,Registered Nurses,"Locker Room, Coatroom, and Dressing Room Atten...",8.1,-65.749149,Registered Nurses,Respiratory Therapists,19.4,-16.345062
2,Registered Nurses,"Office Clerks, General",-5.0,-53.547106,Registered Nurses,"Therapists, All Other",14.6,-28.178207
3,Registered Nurses,Childcare Workers,1.7,-66.940976,Registered Nurses,Exercise Physiologists,11.3,-32.917140
4,Registered Nurses,Proofreaders and Copy Markers,-2.7,-44.551646,Registered Nurses,Speech-Language Pathologists,24.9,7.945516
5,Registered Nurses,License Clerks,NaN,NaN,Registered Nurses,Occupational Therapists,15.9,15.891033
6,Registered Nurses,Insurance Claims Clerks,NaN,NaN,Registered Nurses,Nurse Midwives,11.6,43.303065
7,Registered Nurses,Telephone Operators,-27.9,-51.220204,Registered Nurses,Nurse Practitioners,52.4,49.829739
8,Registered Nurses,Medical Transcriptionists,-2.2,-54.455165,Registered Nurses,Licensed Practical and Licensed Vocational Nurses,9.1,-35.215664
9,Registered Nurses,Gaming Dealers,NaN,NaN,Registered Nurses,Nurse Anesthetists,13.7,138.450624


In [52]:
get_df_comparison(df_concat_exact_difference,df_cps,good_dict_new,'Retail Salespersons',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,Retail Salespersons,"Hosts and Hostesses, Restaurant, Lounge, and C...",8.2,-8.566722,Retail Salespersons,Cashiers,-7.4,-6.342669
1,Retail Salespersons,"Locker Room, Coatroom, and Dressing Room Atten...",8.1,-0.576606,Retail Salespersons,Gambling Change Persons and Booth Cashiers,2.5,1.729819
2,Retail Salespersons,Models,0.5,12.273476,Retail Salespersons,Insurance Sales Agents,5.5,101.729819
3,Retail Salespersons,"Ushers, Lobby Attendants, and Ticket Takers",3.3,-6.919275,Retail Salespersons,"Sales Representatives of Services, Except Adve...",6.0,122.322900
4,Retail Salespersons,Telemarketers,-14.2,4.118616,Retail Salespersons,"Sales and Related Workers, All Other",1.9,26.029654
5,Retail Salespersons,Gaming Dealers,NaN,NaN,Retail Salespersons,"Sales Representatives, Wholesale and Manufactu...",4.1,220.840198
6,Retail Salespersons,"Cooks, Fast Food",-13.4,-6.919275,Retail Salespersons,"Sales Representatives, Wholesale and Manufactu...",0.9,137.314662
7,Retail Salespersons,Teacher Assistants,NaN,NaN,Retail Salespersons,Real Estate Sales Agents,2.3,93.822076
8,Retail Salespersons,"Counter Attendants, Cafeteria, Food Concession...",NaN,NaN,Retail Salespersons,Real Estate Brokers,2.6,136.490939
9,Retail Salespersons,Manicurists and Pedicurists,19.2,2.059308,Retail Salespersons,Receptionists and Information Clerks,3.6,19.028007


In [53]:
get_df_comparison(df_concat_exact_difference,df_cps,good_dict_new,'Janitors and Cleaners, Except Maids and Housekeeping Cleaners',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,"Janitors and Cleaners, Except Maids and Housek...",Models,0.5,3.335861,"Janitors and Cleaners, Except Maids and Housek...",Maids and Housekeeping Cleaners,-0.3,-9.401061
1,"Janitors and Cleaners, Except Maids and Housek...","Locker Room, Coatroom, and Dressing Room Atten...",8.1,-8.491281,"Janitors and Cleaners, Except Maids and Housek...",Landscaping and Groundskeeping Workers,10.1,10.917362
2,"Janitors and Cleaners, Except Maids and Housek...",Shoe and Leather Workers and Repairers,-10.3,7.733131,"Janitors and Cleaners, Except Maids and Housek...","Pesticide Handlers, Sprayers, and Applicators,...",8.3,30.629265
3,"Janitors and Cleaners, Except Maids and Housek...",Maids and Housekeeping Cleaners,-0.3,-9.401061,"Janitors and Cleaners, Except Maids and Housek...","Grounds Maintenance Workers, All Other",6.5,16.982563
4,"Janitors and Cleaners, Except Maids and Housek...",Cleaners of Vehicles and Equipment,3.6,-5.989386,"Janitors and Cleaners, Except Maids and Housek...",Tree Trimmers and Pruners,10.7,45.716452
5,"Janitors and Cleaners, Except Maids and Housek...",Manicurists and Pedicurists,19.2,-6.065201,"Janitors and Cleaners, Except Maids and Housek...",Pest Control Workers,8.5,36.087945
6,"Janitors and Cleaners, Except Maids and Housek...",Slaughterers and Meat Packers,3.3,6.520091,"Janitors and Cleaners, Except Maids and Housek...",Construction Laborers,5.4,34.344200
7,"Janitors and Cleaners, Except Maids and Housek...",Crossing Guards,NaN,NaN,"Janitors and Cleaners, Except Maids and Housek...",Retail Salespersons,-0.6,-7.960576
8,"Janitors and Cleaners, Except Maids and Housek...","Graders and Sorters, Agricultural Products",0.0,-6.444276,"Janitors and Cleaners, Except Maids and Housek...",Childcare Workers,1.7,-11.675512
9,"Janitors and Cleaners, Except Maids and Housek...",Solderers and Brazers,NaN,NaN,"Janitors and Cleaners, Except Maids and Housek...","Laborers and Freight, Stock, and Material Move...",4.2,7.581501


## Drop SD

In [76]:
def difference_drop_SD(df):
    '''
    return adjacency matrix difference. Difference is based on column - row
    zeroed out 
    '''
    dim = df.shape[1]
    ini_array1 = df.values - df.values[:, None]
    std = ini_array1.std(axis=2)
    ini_array1[ini_array1 - ini_array1.mean() < -.5*np.broadcast_to(std[..., None],(968,968,dim))] = 0
#     arr = ini_array1[ini_array1 <0] = 0
    arr = np.sqrt(np.mean(np.power(ini_array1,2),axis =2))
#     return arr

    new_df = pd.concat((df.reset_index()['title'], pd.DataFrame(arr, columns=df.reset_index()['title'])), axis=1)
    return new_df

In [77]:
# [-0.565, -0.255, -0.435, -0.06 , -0.815,  0.   ,  0.   ,  0.   ,
#        -0.13 , -0.56 , -0.375, -1.94 , -1.25 , -0.305, -3.44 , -0.38 ,
#        -0.32 , -0.75 ,  0.   , -1.06 , -0.375, -0.44 , -0.5  ,  0.   ,
#         0.   , -0.75 , -0.815,  0.75 ,  0.19 , -1.625, -1.13 , -0.37 ,
#        -0.06 , -0.315, -0.305]

In [78]:
# test = ini_array1.std(axis=2)
# test
# np.broadcast_to(test[..., None],(968,968,35))

In [79]:
# ini_array1 = df_skills.values - df_skills.values[:, None]
# # ini_array1[ini_array1 - np.std(ini_array1, axis=2)] = 0
# ini_array1[ini_array1 - ini_array1.mean() < -ini_array1.std(axis=2,keepdims = True)] = 0
# # arr = np.sqrt(np.mean(ini_array1,axis = 2))
# ini_array1[0][1]
# np.sqrt(np.mean(np.power(ini_array1,2),axis =2))
# # np.std(ini_array1, axis=2)

In [80]:
df_skills.shape

(968, 35)

In [81]:
drop_abilites = difference_drop_SD(df_ab)
drop_gwas =difference_drop_SD(df_gwas)
drop_skills =difference_drop_SD(df_skills)


drop_skills.index = drop_skills.title 
drop_abilites.index = drop_abilites.title 
drop_gwas.index = drop_gwas.title 

drop_skills = drop_skills.drop(['title'],axis =1)
drop_abilites = drop_abilites.drop(['title'],axis =1)
drop_gwas = drop_gwas.drop(['title'],axis =1)

In [82]:
df_concat_drop = pd.concat([drop_gwas,drop_skills,drop_abilites]).groupby(level =0).mean()

In [83]:
get_df_comparison(df_concat_drop,df_cps,good_dict_new,'Registered Nurses',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,Registered Nurses,Teacher Assistants,NaN,NaN,Registered Nurses,Physical Therapists,18.2,22.020431
1,Registered Nurses,"Office Clerks, General",-5.0,-53.547106,Registered Nurses,Respiratory Therapists,19.4,-16.345062
2,Registered Nurses,License Clerks,NaN,NaN,Registered Nurses,"Therapists, All Other",14.6,-28.178207
3,Registered Nurses,Telephone Operators,-27.9,-51.220204,Registered Nurses,Exercise Physiologists,11.3,-32.917140
4,Registered Nurses,"Locker Room, Coatroom, and Dressing Room Atten...",8.1,-65.749149,Registered Nurses,Speech-Language Pathologists,24.9,7.945516
5,Registered Nurses,Childcare Workers,1.7,-66.940976,Registered Nurses,Occupational Therapists,15.9,15.891033
6,Registered Nurses,Personal Care Aides,NaN,NaN,Registered Nurses,Nurse Midwives,11.6,43.303065
7,Registered Nurses,Gaming Dealers,NaN,NaN,Registered Nurses,Nurse Practitioners,52.4,49.829739
8,Registered Nurses,Proofreaders and Copy Markers,-2.7,-44.551646,Registered Nurses,Licensed Practical and Licensed Vocational Nurses,9.1,-35.215664
9,Registered Nurses,Manicurists and Pedicurists,19.2,-64.841090,Registered Nurses,Nurse Anesthetists,13.7,138.450624


In [84]:
get_df_comparison(df_concat_drop,df_cps,good_dict_new,'Retail Salespersons',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,Retail Salespersons,"Hosts and Hostesses, Restaurant, Lounge, and C...",8.2,-8.566722,Retail Salespersons,Cashiers,-7.4,-6.342669
1,Retail Salespersons,"Locker Room, Coatroom, and Dressing Room Atten...",8.1,-0.576606,Retail Salespersons,Gambling Change Persons and Booth Cashiers,2.5,1.729819
2,Retail Salespersons,"Ushers, Lobby Attendants, and Ticket Takers",3.3,-6.919275,Retail Salespersons,Insurance Sales Agents,5.5,101.729819
3,Retail Salespersons,Models,0.5,12.273476,Retail Salespersons,"Sales Representatives of Services, Except Adve...",6.0,122.322900
4,Retail Salespersons,Telemarketers,-14.2,4.118616,Retail Salespersons,"Sales and Related Workers, All Other",1.9,26.029654
5,Retail Salespersons,Gaming Dealers,NaN,NaN,Retail Salespersons,"Sales Representatives, Wholesale and Manufactu...",4.1,220.840198
6,Retail Salespersons,"Cooks, Fast Food",-13.4,-6.919275,Retail Salespersons,"Sales Representatives, Wholesale and Manufactu...",0.9,137.314662
7,Retail Salespersons,"Counter Attendants, Cafeteria, Food Concession...",NaN,NaN,Retail Salespersons,Real Estate Sales Agents,2.3,93.822076
8,Retail Salespersons,Teacher Assistants,NaN,NaN,Retail Salespersons,Real Estate Brokers,2.6,136.490939
9,Retail Salespersons,Manicurists and Pedicurists,19.2,2.059308,Retail Salespersons,Receptionists and Information Clerks,3.6,19.028007


In [85]:
get_df_comparison(df_concat_drop,df_cps,good_dict_new,'Janitors and Cleaners, Except Maids and Housekeeping Cleaners',10)

,ONET_occ_before,ONET_occ_after,ONET_employment_proj,ONET_wage_change,CPS_occ_before,CPS_occ_after,CPS_employment_proj,CPS_wage_change
0,"Janitors and Cleaners, Except Maids and Housek...",Models,0.5,3.335861,"Janitors and Cleaners, Except Maids and Housek...",Maids and Housekeeping Cleaners,-0.3,-9.401061
1,"Janitors and Cleaners, Except Maids and Housek...","Locker Room, Coatroom, and Dressing Room Atten...",8.1,-8.491281,"Janitors and Cleaners, Except Maids and Housek...",Landscaping and Groundskeeping Workers,10.1,10.917362
2,"Janitors and Cleaners, Except Maids and Housek...",Shoe and Leather Workers and Repairers,-10.3,7.733131,"Janitors and Cleaners, Except Maids and Housek...","Pesticide Handlers, Sprayers, and Applicators,...",8.3,30.629265
3,"Janitors and Cleaners, Except Maids and Housek...",Maids and Housekeeping Cleaners,-0.3,-9.401061,"Janitors and Cleaners, Except Maids and Housek...","Grounds Maintenance Workers, All Other",6.5,16.982563
4,"Janitors and Cleaners, Except Maids and Housek...",Cleaners of Vehicles and Equipment,3.6,-5.989386,"Janitors and Cleaners, Except Maids and Housek...",Tree Trimmers and Pruners,10.7,45.716452
5,"Janitors and Cleaners, Except Maids and Housek...",Manicurists and Pedicurists,19.2,-6.065201,"Janitors and Cleaners, Except Maids and Housek...",Pest Control Workers,8.5,36.087945
6,"Janitors and Cleaners, Except Maids and Housek...",Slaughterers and Meat Packers,3.3,6.520091,"Janitors and Cleaners, Except Maids and Housek...",Construction Laborers,5.4,34.344200
7,"Janitors and Cleaners, Except Maids and Housek...",Crossing Guards,NaN,NaN,"Janitors and Cleaners, Except Maids and Housek...",Retail Salespersons,-0.6,-7.960576
8,"Janitors and Cleaners, Except Maids and Housek...",Solderers and Brazers,NaN,NaN,"Janitors and Cleaners, Except Maids and Housek...",Childcare Workers,1.7,-11.675512
9,"Janitors and Cleaners, Except Maids and Housek...","Graders and Sorters, Agricultural Products",0.0,-6.444276,"Janitors and Cleaners, Except Maids and Housek...","Laborers and Freight, Stock, and Material Move...",4.2,7.581501
